In [2]:
#ノック４１

import pandas as pd
customer = pd.read_csv('data/customer_join.csv')
uselog_months = pd.read_csv('data/use_log_months.csv')

year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del  tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [9]:
#ノック４２
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

C:\Users\user\AppData\Local\Temp/ipykernel_13508/2595789816.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"] = None
C:\Users\user\AppData\Local\Temp/ipykernel_13508/2595789816.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])


33851
1104
1104


C:\Users\user\AppData\Local\Temp/ipykernel_13508/2595789816.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["年月"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [13]:
#ノック４３

conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

33851
27422
2842
3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201902,IK755649,4,3.0,XXXXX,C01,M,2015-12-01,NaN,CA1,...,10500.0,通常,4.750000,5.0,8.0,3.0,1.0,2019-04-30,40.0,NaN
1,201811,TS722194,4,5.0,XXXX,C03,M,2016-09-01,NaN,CA1,...,6000.0,通常,5.000000,5.0,6.0,3.0,1.0,2019-04-30,31.0,NaN
2,201807,OA953837,6,4.0,XXXX,C03,M,2016-04-01,NaN,CA1,...,6000.0,通常,4.500000,4.0,8.0,2.0,1.0,2019-04-30,36.0,NaN
3,201806,AS662285,6,8.0,XXXXX,C01,M,2017-04-01,NaN,CA1,...,10500.0,通常,5.916667,6.0,8.0,3.0,1.0,2019-04-30,24.0,NaN
4,201808,TS875601,6,8.0,XXXX,C03,M,2017-09-01,NaN,CA1,...,6000.0,通常,6.166667,6.0,10.0,3.0,1.0,2019-04-30,19.0,NaN


In [15]:
#ノック４４

predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

C:\Users\user\AppData\Local\Temp/ipykernel_13508/1189034777.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,new_date,now_date
0,201902,IK755649,4,3.0,XXXXX,C01,M,2015-12-01,NaN,CA1,...,5.0,8.0,3.0,1.0,2019-04-30,40.0,NaN,38,2019-02-01,2019-02-01
1,201811,TS722194,4,5.0,XXXX,C03,M,2016-09-01,NaN,CA1,...,5.0,6.0,3.0,1.0,2019-04-30,31.0,NaN,26,2018-11-01,2018-11-01
2,201807,OA953837,6,4.0,XXXX,C03,M,2016-04-01,NaN,CA1,...,4.0,8.0,2.0,1.0,2019-04-30,36.0,NaN,27,2018-07-01,2018-07-01
3,201806,AS662285,6,8.0,XXXXX,C01,M,2017-04-01,NaN,CA1,...,6.0,8.0,3.0,1.0,2019-04-30,24.0,NaN,14,2018-06-01,2018-06-01
4,201808,TS875601,6,8.0,XXXX,C03,M,2017-09-01,NaN,CA1,...,6.0,10.0,3.0,1.0,2019-04-30,19.0,NaN,11,2018-08-01,2018-08-01


In [17]:
#ノック４５

predict_data.isna().sum()

predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2648
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2648
period                  0
new_date                0
now_date                0
dtype: int64

In [19]:
#ノック４６

target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

predict_data = pd.get_dummies(predict_data)
predict_data.head()

del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,3.0,1.0,38,0.0,0,0,1,1,0,0,0,1
1,5.0,1.0,26,0.0,0,0,1,0,0,1,0,1
2,4.0,1.0,27,0.0,0,0,1,0,0,1,0,1
3,8.0,1.0,14,0.0,0,0,1,1,0,0,0,1
4,8.0,1.0,11,0.0,0,0,1,0,0,1,0,1


In [22]:
#ノック４７

#決定木を用いた予測モデル
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

[1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1.
 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0.
 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0.
 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1.
 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1.

,y_test,y_pred
2017,0.0,1.0
1214,0.0,0.0
1089,0.0,0.0
1296,0.0,0.0
1725,0.0,0.0


In [24]:
#ノック４８

correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

#決定木の深さを狭めることでモデルを簡易化し、過学習を抑え、未知のデータにも対応できやすくする
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8954372623574145
0.8954372623574145
0.9816223067173637
0.9125475285171103
0.9296577946768061


In [25]:
#ノック４９

importance = pd.DataFrame({"feature_names":X.columns, "coeffcient":model.feature_importances_})
importance

,feature_names,coeffcient
0,count_1,0.342581
1,routine_flg,0.117778
2,period,0.539641
3,campaign_name_入会費半額,0.000000
4,campaign_name_入会費無料,0.000000
5,campaign_name_通常,0.000000
6,class_name_オールタイム,0.000000
7,class_name_デイタイム,0.000000
8,class_name_ナイト,0.000000
9,gender_F,0.000000
